In [4]:
!pip install boto3


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import boto3
 
# Especificar la región y las claves de acceso AWS
aws_region = 'eu-west-3'
aws_access_key_id = ''
aws_secret_access_key = ''
 
# Crear un cliente de DynamoDB con la región y claves especificadas
client = boto3.client(
    'dynamodb',
    region_name=aws_region,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)
 
# Crear un recurso de DynamoDB con la región y claves especificadas
dynamodb = boto3.resource(
    'dynamodb',
    region_name=aws_region,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

In [ ]:

import boto3
from collections import defaultdict
from boto3.dynamodb.conditions import Attr, Key

ordenes = dynamodb.Table('Ordenes')
productos = dynamodb.Table('productos')

# Realizamos un scan para obtener todos los ítems de la tabla de productos
response_productos = productos.scan()
items_productos = response_productos['Items']

# Realizamos un scan para obtener todos los ítems de la tabla de órdenes
response_ordenes = ordenes.scan()
items_ordenes = response_ordenes['Items']
 
# Verificamos los datos obtenidos
print("Productos obtenidos:")
print(items_productos)
print("Órdenes obtenidas:")
print(items_ordenes)

In [ ]:

import decimal
if items_ordenes:
    seen_product_ids = set()
    
    highest_rated_products = []
    for item in items_ordenes:
        if item['Rating'] == decimal.Decimal('5') and item['ProductID'] not in seen_product_ids:
            highest_rated_products.append(item)
            seen_product_ids.add(item['ProductID'])

    print(" Productos con mayor rating:")
    if highest_rated_products:
        for product in highest_rated_products:
            print(f" ProductID: {product['ProductID']}")
            print(f" Rating: {product['Rating']}")
            print(f" Nombre del Cliente: {product['NameClient']}")
            print(f" OrderID: {product['OrderID']}")
            print("----")
    else:
        print("No hay productos con rating 5.")
else:
    print("No hay productos en la tabla.")


In [ ]:

import decimal
if items_ordenes:
 
    seen_product_ids = set()
    
    highest_rated_products = []
    for item in items_ordenes:
        if item['Rating'] == decimal.Decimal('1') and item['ProductID'] not in seen_product_ids:
            highest_rated_products.append(item)
            seen_product_ids.add(item['ProductID'])

    print(" Productos con mayor rating:")
    if highest_rated_products:
        for product in highest_rated_products:
            print(f" ProductID: {product['ProductID']}")
            print(f" Rating: {product['Rating']}")
            print(f" Nombre del Cliente: {product['NameClient']}")
            print(f" OrderID: {product['OrderID']}")
            print("----")
    else:
        print("No hay productos con rating 1.")
else:
    print("No hay productos en la tabla.")


In [ ]:
# Ventas por ubicación
# Consulta: Obtener las ventas totales por ubicación (ciudad, región, etc.).
# Objetivo: Identificar qué áreas geográficas están generando más ventas.

In [6]:
from collections import defaultdict

sales_by_location = defaultdict(int)

for item in items_ordenes:
    if 'OrderLocation' in item and 'Quantity' in item:
        location = item['OrderLocation']
        quantity = int(item['Quantity']) 
        sales_by_location[location] += quantity

sorted_sales_by_location = sorted(sales_by_location.items(), key=lambda x: x[1], reverse=True)

print(" Cantidad de ventas por lugar :")
for location, total_sales in sorted_sales_by_location:
    print(f" {location}: {total_sales} ventas")


 Cantidad de ventas por lugar :
 San Francisco: 352 ventas
 Sacramento: 345 ventas
 Las Vegas: 335 ventas
 New York: 322 ventas
 Cleveland: 316 ventas
 Portland: 312 ventas
 New Jersey: 306 ventas
 San Antonio: 300 ventas
 Pittsburgh: 298 ventas
 Memphis: 286 ventas
 Austin: 283 ventas
 Detroit: 279 ventas
 Miami: 266 ventas
 London: 163 ventas
 Montreal: 161 ventas
 Dublin: 156 ventas
 Manchester: 155 ventas
 Liverpool: 150 ventas
 Mumbai: 148 ventas
 Cardiff: 147 ventas
 Paris: 147 ventas
 Sydney: 140 ventas
 Glasgow: 135 ventas
 Toronto: 133 ventas
 New Delhi: 130 ventas


In [38]:
# Ventas por categoría de producto
# Consulta: Obtener el total de ventas por categoría (por ejemplo, 'Prenda', 'Electrónica', etc.).
# Objetivo: Determinar qué categorías de productos son más populares y contribuyen más a las ventas.

In [7]:

category_sales = defaultdict(int)

for item_orden in items_ordenes:
    if 'ProductID' in item_orden and 'Quantity' in item_orden:
        product_id = item_orden['ProductID']
        quantity = int(item_orden['Quantity'])

        product_info = next((prod for prod in items_productos if prod.get('ProductID') == product_id), None)

        if product_info:
            category = product_info.get('categoria', 'Desconocida')

            category_sales[category] += quantity
        else:
            print(f"Producto con ProductID {product_id} no encontrado en la lista de productos.")

sorted_category_sales = sorted(category_sales.items(), key=lambda x: x[1], reverse=True)

print(" Ventas por categoría de producto:")
for category, total_sales in sorted_category_sales:
    print(f" Categoría: {category} - Total Ventas: {total_sales}")


 Ventas por categoría de producto:
 Categoría: Prenda - Total Ventas: 2912
 Categoría: Accesorio - Total Ventas: 2853


In [40]:
# Promedio de ventas por cliente
# Consulta: Obtener el promedio de ventas por cliente.
# Objetivo: Evaluar el comportamiento de los clientes y su nivel de gasto promedio.

In [8]:
from collections import defaultdict

customer_sales = defaultdict(int)

for item in items_ordenes:
    if 'ClientID' in item and 'TotalSales' in item:
        client_id = item['ClientID']
        customer_sales[client_id] += float(item['TotalSales'])

total_customers = len(customer_sales)
total_sales = sum(customer_sales.values())
avg_sales_per_customer = total_sales / total_customers if total_customers else 0

print(f" Promedio de ventas por cliente: {avg_sales_per_customer:.2f}")

 Promedio de ventas por cliente: 576.01


In [43]:
# Porcentaje de productos con calificación alta (Rating >= 4)
# Consulta: Obtener el porcentaje de productos con una calificación de 4 o más.
# Objetivo: Medir la satisfacción general de los clientes y el rendimiento de los productos.

In [9]:
import decimal

high_rated_count = 0
total_products = len(items_ordenes)

for item in items_ordenes:
    if 'Rating' in item and float(item['Rating']) >= 4:  # Acceder directamente a 'Rating'
        high_rated_count += 1

high_rating_percentage = (high_rated_count / total_products) * 100 if total_products else 0

print(f"Porcentaje de productos con rating 4 o más: {high_rating_percentage:.2f}%")


Porcentaje de productos con rating 4 o más: 60.48%


In [ ]:
# Ventas por cantidad de productos
# Consulta: Obtener el total de ventas por cantidad de productos vendidos.
# Objetivo: Saber qué tan rápido se venden los productos en comparación con otros.

In [10]:

from collections import defaultdict

product_quantity_sales = defaultdict(int)

for item in items_ordenes:
    if 'ProductID' in item and 'Quantity' in item:
        product_id = item['ProductID']
        product_quantity_sales[product_id] += int(item['Quantity'])

print("Ventas por cantidad de productos:")
for product_id, quantity in product_quantity_sales.items():
    print(f"Producto {product_id}: {quantity} unidades vendidas")


Ventas por cantidad de productos:
Producto BF2370: 234 unidades vendidas
Producto BF1414: 264 unidades vendidas
Producto BF8063: 261 unidades vendidas
Producto BF1681: 249 unidades vendidas
Producto BF0103: 230 unidades vendidas
Producto BF9289: 260 unidades vendidas
Producto BF8238: 256 unidades vendidas
Producto BF6693: 236 unidades vendidas
Producto BF2887: 226 unidades vendidas
Producto BF6975: 252 unidades vendidas
Producto BF2152: 206 unidades vendidas
Producto BF9878: 274 unidades vendidas
Producto BF0060: 257 unidades vendidas
Producto BF6167: 244 unidades vendidas
Producto BF3942: 238 unidades vendidas
Producto BF7814: 261 unidades vendidas
Producto BF5238: 231 unidades vendidas
Producto BF9724: 270 unidades vendidas
Producto BF0809: 197 unidades vendidas
Producto BF6604: 235 unidades vendidas
Producto BF1772: 260 unidades vendidas
Producto BF6885: 222 unidades vendidas
Producto BF4831: 170 unidades vendidas
Producto BF9708: 232 unidades vendidas


In [ ]:
# Análisis de ventas por género del comprador
# Consulta: Obtener las ventas totales por género de los compradores.
# Objetivo: Analizar las diferencias en las preferencias de compra según el género

In [11]:
import decimal
from collections import defaultdict

gender_sales = defaultdict(int)

for item in items_ordenes:
    if 'BuyerGender' in item and 'TotalSales' in item:
        # 'TotalSales' ya es un Decimal, lo convertimos directamente a float
        gender_sales[item['BuyerGender']] += float(item['TotalSales'])

print(" Ventas por género del comprador:")
for gender, total_sales in gender_sales.items():
    print(f" Género: {gender} - Ventas Totales: {total_sales}")



 Ventas por género del comprador:
 Género: Female - Ventas Totales: 113590.0
 Género: Male - Ventas Totales: 287311.0


In [ ]:
# Análisis de ventas por fecha (últimos 100 días, por ejemplo)
# Consulta: Obtener las ventas realizadas en los últimos 30 días, mostrando el total de ventas por día.
# Objetivo: Analizar el comportamiento de ventas y detectar tendencias o patrones estacionales.

In [12]:
from datetime import datetime, timedelta
from collections import defaultdict

today = datetime.today()
thirty_days_ago = today - timedelta(days=100)

daily_sales = defaultdict(float)

for item in items_ordenes:
    if 'OrderDate' in item and 'TotalSales' in item:
        order_date = datetime.strptime(item['OrderDate'], '%d/%m/%Y')
        if order_date >= thirty_days_ago:
            daily_sales[order_date.date()] += float(item['TotalSales'])

sorted_daily_sales = sorted(daily_sales.items(), key=lambda x: x[0])

print(" Ventas por fecha en los últimos 30 días:")
for date, total_sales in sorted_daily_sales:
    print(f" Fecha: {date} - Ventas Totales: {total_sales}")



 Ventas por fecha en los últimos 30 días:
 Fecha: 2024-10-23 - Ventas Totales: 354.0
 Fecha: 2024-10-24 - Ventas Totales: 959.0
 Fecha: 2024-10-25 - Ventas Totales: 750.0
 Fecha: 2024-10-26 - Ventas Totales: 999.0
 Fecha: 2024-10-27 - Ventas Totales: 739.0
 Fecha: 2024-10-28 - Ventas Totales: 612.0
 Fecha: 2024-10-29 - Ventas Totales: 573.0
 Fecha: 2024-10-30 - Ventas Totales: 1384.0
 Fecha: 2024-10-31 - Ventas Totales: 1353.0
 Fecha: 2024-11-01 - Ventas Totales: 2133.0
 Fecha: 2024-11-02 - Ventas Totales: 463.0
 Fecha: 2024-11-03 - Ventas Totales: 1314.0
 Fecha: 2024-11-04 - Ventas Totales: 523.0


In [ ]:
# Promedio de ventas por ubicación y categoría
# Consulta: Obtener el promedio de ventas por ubicación y categoría de producto.
# Objetivo: Analizar qué combinaciones de ubicación y categoría generan más ventas en promedio.

In [ ]:
location_category_sales = defaultdict(lambda: defaultdict(float))
location_category_count = defaultdict(lambda: defaultdict(int))

for item in items_ordenes:
    if 'OrderLocation' in item and 'ProductID' in item and 'TotalSales' in item:
        location = item['OrderLocation']
        product_id = item['ProductID']  
        
        product_info = next((prod for prod in items_productos if prod['ProductID'] == product_id), None) 
        if product_info:
            category = product_info['categoria']  # Ya no usamos ['S']
            location_category_sales[location][category] += float(item['TotalSales'])
            location_category_count[location][category] += 1

for location in location_category_sales:
    for category in location_category_sales[location]:
        total_sales = location_category_sales[location][category]
        total_count = location_category_count[location][category]
        avg_sales = total_sales / total_count if total_count else 0
        print(f" Ubicación: {location}, Categoría: {category} - Promedio de Ventas: {avg_sales:.2f}")



In [ ]:
# Productos con mayor ingreso total
# Consulta: Obtener los productos que generaron más ingresos en total.
# Objetivo: Identificar qué productos están generando los mayores ingresos y ajustar estrategias de marketing o inventario.

In [17]:

import decimal
product_revenue = defaultdict(float)

for item in items_ordenes:
    if 'ProductID' in item and 'TotalSales' in item and 'Quantity' in item:
        product_id = item['ProductID']  
        revenue = float(item['TotalSales'])  
        product_revenue[product_id] += revenue

sorted_product_revenue = sorted(product_revenue.items(), key=lambda x: x[1], reverse=True)

print(" Productos con mayor ingreso total:")
for product_id, total_revenue in sorted_product_revenue:
    print(f" ProductoID: {product_id} - Ingreso Total: {total_revenue:.2f}")



 Productos con mayor ingreso total:
 ProductoID: BF9878 - Ingreso Total: 20009.00
 ProductoID: BF1772 - Ingreso Total: 19514.00
 ProductoID: BF9289 - Ingreso Total: 19055.00
 ProductoID: BF9724 - Ingreso Total: 18790.00
 ProductoID: BF1414 - Ingreso Total: 18665.00
 ProductoID: BF6167 - Ingreso Total: 18636.00
 ProductoID: BF6604 - Ingreso Total: 18315.00
 ProductoID: BF7814 - Ingreso Total: 18083.00
 ProductoID: BF1681 - Ingreso Total: 18041.00
 ProductoID: BF0060 - Ingreso Total: 17611.00
 ProductoID: BF8063 - Ingreso Total: 17303.00
 ProductoID: BF3942 - Ingreso Total: 17243.00
 ProductoID: BF6693 - Ingreso Total: 17173.00
 ProductoID: BF8238 - Ingreso Total: 16870.00
 ProductoID: BF5238 - Ingreso Total: 15927.00
 ProductoID: BF6975 - Ingreso Total: 15771.00
 ProductoID: BF2887 - Ingreso Total: 15647.00
 ProductoID: BF9708 - Ingreso Total: 14962.00
 ProductoID: BF2370 - Ingreso Total: 14945.00
 ProductoID: BF2152 - Ingreso Total: 14706.00
 ProductoID: BF6885 - Ingreso Total: 14402.0

In [ ]:
# Ventas según el canal de envío (nacional vs internacional)
# Consulta: Obtener las ventas totales según el tipo de envío (nacional o internacional).
# Objetivo: Evaluar si los envíos internacionales están generando un porcentaje significativo de las ventas.

In [18]:
shipping_sales = defaultdict(float)

for item in items_ordenes:
    if 'InternationalShipping' in item and 'TotalSales' in item:
        shipping_type = item['InternationalShipping']
        

        if shipping_type == 'Yes':
            shipping_type = 'Internacional'
        elif shipping_type == 'No':
            shipping_type = 'Nacional'
        
        shipping_sales[shipping_type] += float(item['TotalSales'])

print(" Ventas por tipo de envío:")
for shipping_type, total_sales in shipping_sales.items():
    print(f" Envío {shipping_type}: Ventas Totales: {total_sales:.2f}")


 Ventas por tipo de envío:
 Envío Internacional: Ventas Totales: 182155.00
 Envío Nacional: Ventas Totales: 218746.00


In [ ]:
# Clientes recurrentes
# Consulta: Obtener la cantidad de clientes recurrentes (que han hecho más de 10 compras).
# Objetivo: Identificar cuántos clientes están regresando y la fidelidad de la base de clientes.

In [26]:
from collections import defaultdict

clientes_compras = defaultdict(int)
clientes_nombres = {}

for item in items_ordenes:
    if 'ClientID' in item and 'NameClient' in item:
        clientes_compras[item['ClientID']] += 1
        clientes_nombres[item['ClientID']] = item['NameClient'] 

clientes_recurrentes = {cliente: compras for cliente, compras in clientes_compras.items() if compras > 10}
cantidad_recurrentes = len(clientes_recurrentes)

print(f" Clientes Recurrentes: {cantidad_recurrentes}")
for cliente, compras in clientes_recurrentes.items():
    nombre_cliente = clientes_nombres.get(cliente, "Desconocido")  # Obtener el nombre o mostrar "Desconocido" si falta
    print(f" Cliente: {nombre_cliente} - Compras Realizadas: {compras}")


 Clientes Recurrentes: 10
 Cliente: Adele Dwane - Compras Realizadas: 12
 Cliente: Christine Jacquest - Compras Realizadas: 12
 Cliente: Jacklyn Rudall - Compras Realizadas: 13
 Cliente: Stevena Guillain - Compras Realizadas: 12
 Cliente: Bonni Bliven - Compras Realizadas: 11
 Cliente: Erin Flageul - Compras Realizadas: 12
 Cliente: Bret Whyler - Compras Realizadas: 11
 Cliente: Serge Stevani - Compras Realizadas: 11
 Cliente: Mattias Hassin - Compras Realizadas: 11
 Cliente: Clarette Jerred - Compras Realizadas: 11
